# Plant Disease Detection Project 🌿

## Step 1: Data Loading and Exploration

In [1]:
import os

# --- 1. ENVIRONMENT CONFIGURATION (Must be set before importing TensorFlow) ---
# Set logging level to reduce unnecessary warnings (0 = all, 1 = no info, 2 = no warnings)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Prevent kernel crashes by allowing memory to grow as needed instead of allocating all VRAM at once
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

print("--- System Test ---")
print(f"TensorFlow Version: {tf.__version__}")

# --- 2. GPU DETECTION & MEMORY GROWTH PROTOCOL ---
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable incremental memory growth for the first detected GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"✅ SUCCESS: GPU Detected and Enforced -> {gpus[0]}")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(f"⚠️ GPU Runtime Error: {e}")
else:
    print("❌ ERROR: No GPU detected. Check WSL CUDA drivers or library paths.")

# List all detected physical devices for confirmation
print("Detected Devices:", tf.config.list_physical_devices())

# --- 3. DATASET PATH & CLASS VERIFICATION ---
# Direct path to the 'color' folder containing the 38 plant disease classes
base_path = "/home/youssef/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3/plantvillage dataset/color"

print("\n--- Dataset Verification ---")
if os.path.exists(base_path):
    # Retrieve and sort class names to ensure consistency
    all_classes = os.listdir(base_path)
    all_classes.sort()
    
    print(f"✅ Success! Path exists.")
    print(f"✅ Total Classes Found: {len(all_classes)}")
    
    # Display the first 10 classes as a sample check
    print("Sample Classes (First 10):", all_classes[:10])
else:
    print(f"❌ Critical Error: Path not found at: {base_path}")
    
    # Debugging: Check the parent directory to find the correct structure
    parent_path = "/home/youssef/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3/plantvillage dataset"
    if os.path.exists(parent_path):
        print("Available directories in parent path:", os.listdir(parent_path))
    else:
        print("❌ Parent directory is also missing. Please verify the Kagglehub download path.")


--- System Test ---
TensorFlow Version: 2.20.0
✅ SUCCESS: GPU Detected and Enforced -> PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Detected Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

--- Dataset Verification ---
✅ Success! Path exists.
✅ Total Classes Found: 38
Sample Classes (First 10): ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight']


In [ ]:
import tensorflow as tf
import sys

print("--- Deep Diagnostic ---")

print("Is built with CUDA:", tf.test.is_built_with_cuda())
print("Is built with GPU support:", tf.test.is_built_with_gpu_support())

try:
    from tensorflow.python.client import device_lib
    print(device_lib.list_local_devices())
except Exception as e:
    print(f"Detailed Error: {e}")

## step 2: Exploratory Data Analysis 

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

# 1. Get a list of all categories (folders)
subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

plt.figure(figsize=(16, 16))

# 2. Choose 16 random images
for i in range(16):
    # Choose a random folder
    random_folder = random.choice(subfolders)
    folder_name = os.path.basename(random_folder)
    
    # Select a random image from inside the folder
    random_image = random.choice(os.listdir(random_folder))
    img_path = os.path.join(random_folder, random_image)
    
    # Open and display the image
    img = Image.open(img_path)
    
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(img)
    plt.title(folder_name, fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()

# 1. Observation: High image quality but controlled environment (Lab-like).
# 2. Risk: Model might overfit on plain backgrounds and fail in real farm conditions.
# 3. Decision: We MUST implement Heavy Data Augmentation (Rotation, Brightness,shearing,Zoom) 
#    during the Data Preparation phase to bridge the gap with real-world noise.
# 4. Imbalance Check: If some classes have significantly fewer images, 
#    Augmentation will also be used to balance the dataset.


In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Prepare data for plotting
classes = []
counts = []

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        classes.append(folder)
        # Count number of files in the directory
        counts.append(len(os.listdir(folder_path)))

# 2. Create a DataFrame for easier handling
df = pd.DataFrame({'Class': classes, 'Count': counts})

# 3. Sort by count to see the distribution clearly
df = df.sort_values(by='Count', ascending=False)

# 4. Visualization
plt.figure(figsize=(12, 10))
sns.set_style("whitegrid")
plot = sns.barplot(data=df, y='Class', x='Count', palette='viridis')

# 5. Add count labels to each bar
for i in plot.containers:
    plot.bar_label(i, padding=5)

plt.title('Distribution of Images per Class in PlantVillage Dataset', fontsize=15)
plt.xlabel('Number of Images', fontsize=12)
plt.ylabel('Disease Class', fontsize=12)
plt.tight_layout()
plt.show()

# 6. Check for imbalance
min_count = df['Count'].min()
max_count = df['Count'].max()
print(f"--- Dataset Statistics ---")
print(f"Total Classes: {len(df)}")
print(f"Total Images: {df['Count'].sum()}")
print(f"Min Images in a class: {min_count}")
print(f"Max Images in a class: {max_count}")
print(f"Imbalance Ratio (Max/Min): {max_count / min_count:.2f}")

# --- FINAL EDA CONCLUSIONS ---

# 1. OBSERVATION: Severe class imbalance detected (Ratio 36.23:1).
# 2. RISK: The model will be highly biased towards majority classes (e.g., Orange, Tomato Yellow Leaf Curl).
# 3. STRATEGY: 
#    - Use "Oversampling" via Heavy Data Augmentation for minority classes (Potato healthy, Apple rust).
#    - This will help the model recognize rare diseases as effectively as common ones.
# 4. REAL-WORLD GAP: 
#    - Since current images are "clean," Augmentation (Noise, Blur, Brightness) 
#    - is critical to simulate field conditions and prevent overfitting on lab backgrounds.

# Next Phase: Data Splitting and Image Preprocessing (including Augmentation layers).

## step 3: Data Splitting


In [2]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

base_path = "/home/youssef/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3/plantvillage dataset/color"

# Define the paths for the new split folders
base_dir = '/home/youssef/plant_disease_split' # Path to store the split data
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Loop through each disease class and split its images
for class_name in os.listdir(base_path):
    class_path = os.path.join(base_path, class_name)
    if os.path.isdir(class_path):
        # Get all images in this class
        images = os.listdir(class_path)
        
        # Split: 80% Train, 20% Temp (which will be split into Val and Test)
        train_imgs, temp_imgs = train_test_split(images, test_size=0.2, random_state=42)
        
        # Split Temp: 50% Val, 50% Test (resulting in 10% Val and 10% Test of total)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)
        
        # Helper function to copy files to their respective folders
        def copy_images(img_list, source, destination):
            dest_class_path = os.path.join(destination, class_name)
            if not os.path.exists(dest_class_path):
                os.makedirs(dest_class_path)
            for img_name in img_list:
                shutil.copy(os.path.join(source, img_name), os.path.join(dest_class_path, img_name))

        # Copying files... this might take a minute
        copy_images(train_imgs, class_path, train_dir)
        copy_images(val_imgs, class_path, val_dir)
        copy_images(test_imgs, class_path, test_dir)

print("✅ Success! Data is now split into Train (80%), Val (10%), and Test (10%).")

✅ Success! Data is now split into Train (80%), Val (10%), and Test (10%).


## step 4: Image Processing

In [ ]:
import tensorflow as tf

# 1. Configuration Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# 2. Loading and Preprocessing Training Data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical' # Important for multi-class (38 classes) One-Hot Encoding
)

# 3. Loading Validation and Test Data
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# 4. Normalization Layer (The "Scaling" Step)
# This scales pixels from [0, 255] to [0, 1]
from tensorflow.keras.applications.resnet50 import preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))
test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

# 5. Prefetching (Performance Boost)
# This allows the CPU to prepare the next batch while the GPU is working
# 4. Performance Optimization
    # الـ shuffle هنا للـ Train بس
    # 1. الـ Shuffle: خليه 100 لو الرام تسمح، لو خايف خليها 50 (أفضل من 20 عشان التنوع)
train_ds = train_ds.shuffle(buffer_size=100) 
    
    # 2. الـ Prefetch: أهم تعديل، هنخليه يحضر 2 batches بس في الذاكرة
    # ده بيمنع الـ Terminal من حجز مساحة صور عملاقة في الرام
train_ds = train_ds.prefetch(buffer_size=2)
val_ds = val_ds.prefetch(buffer_size=2)
test_ds = test_ds.prefetch(buffer_size=2)
print("✅ Preprocessing Complete! Datasets are normalized and batched.")



Found 43429 files belonging to 38 classes.


I0000 00:00:1771805072.361455   17229 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3536 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 5428 files belonging to 38 classes.
Found 5448 files belonging to 38 classes.
✅ Preprocessing Complete! Datasets are normalized and batched.


### Check

In [ ]:
# Get one batch from the training set
for images, labels in train_ds.take(1):
    print(f"Image shape: {images[0].shape}")
    print(f"Max pixel value: {tf.reduce_max(images[0])}")
    print(f"Min pixel value: {tf.reduce_min(images[0])}")
    break

## step 5 : Data Augmentation

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

# 1. Define only the augmentation logic
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
], name="heavy_augmentation_layer")

# 2. Update ONLY the training dataset with augmentation
# Note: Since normalization is already done in your previous cell, 
# we just apply augmentation here.
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# 3. Re-apply prefetch to ensure best performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

print("✅ Data Augmentation successfully added to the existing pipeline!")

print("✅ Professional Pipeline Complete!")
print("  - Train set: Normalized + Heavy Augmentation (For robust learning)")
print("  - Val/Test sets: Normalized only (For fair evaluation)")

✅ Data Augmentation successfully added to the existing pipeline!
✅ Professional Pipeline Complete!
  - Train set: Normalized + Heavy Augmentation (For robust learning)
  - Val/Test sets: Normalized only (For fair evaluation)


## step 6: Building Model 

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# --- 1. GPU ENFORCEMENT PROTOCOL ---
# Verify the presence of the GPU and force the TensorFlow to use it.
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Allowing memory to grow gradually so that all VRAM is not reserved at once (Memory Growth)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Set GPU #0 as the default processing device
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"✅ GPU is detected and ENFORCED: {gpus[0]}")
    except RuntimeError as e:
        print(f"❌ GPU Error: {e}")
else:
    print("⚠️ No GPU detected. Check your WSL CUDA drivers.")

# Ensure all model operations and training happen on the GPU
with tf.device('/GPU:0'):
    
    # --- 1. MODEL ARCHITECTURE ---
    # Load ResNet50 as the base model
    # weights='imagenet': use pre-trained weights
    # include_top=False: remove the original 1000-class classification head
    base_model = tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # Freeze the Base Model to protect pre-trained weights during initial training
    base_model.trainable = False

    # Define the custom head for our specific 38-class task
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  # Reduces spatial dimensions to a feature vector
        layers.BatchNormalization(),       # Improves training stability and speed
        layers.Dense(256, activation='relu'), # Intermediate dense layer for feature extraction
        layers.Dropout(0.4),               # Regularization to prevent overfitting
        layers.Dense(38, activation='softmax') # Final output layer for 38 categories
    ])

    # Compile the Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # --- 2. CALLBACKS SETTINGS ---
    # Stop training if validation loss stops improving
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Dynamically reduce learning rate when the model hits a plateau
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=4,
        min_lr=1e-6,
        verbose=1
    )

    # Automatically save the best model iteration
    checkpoint = ModelCheckpoint(
        'best_plant_disease_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )

    callbacks = [early_stopping, reduce_lr, checkpoint]

    # --- 4. DATASET OPTIMIZATION ---
    # شلنا الـ unbatch التقيل وضفنا repeat عشان نتجنب الـ Out of range error

    NEW_BATCH_SIZE = 32
    train_ds = train_ds.unbatch().batch(NEW_BATCH_SIZE).repeat().prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.unbatch().batch(NEW_BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # حساب الـ steps عشان الـ repeat ما تعملش Infinite loop
    STEPS_PER_EPOCH = 1697      # 54305 // 32

    # --- 5. START STAGE 1 (Training the Head) ---
    print("🚀 Stage 1: Training the Custom Head (Frozen Base)...")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10, # 10 epochs كفاية جداً لتسخين الـ Head
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=callbacks,
        verbose=1
    )

    # --- 6. START STAGE 2 (Fine-Tuning) ---
    # 
    print("🔓 Stage 2: Unfreezing Base Model for Fine-Tuning...")
    base_model.trainable = True
    
    # تجميد أول 100 طبقة وفتح الباقي (الطبقات العليا هي اللي بتفهم تفاصيل النباتات)
    for layer in base_model.layers[:-50]:
        layer.trainable = False

    # إعادة الـ Compile بـ Learning Rate "بطيء جداً"
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("🚀 Fine-tuning initiated on GPU...")
    history_fine = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=40, # بنكمل الـ 50 epoch بتوعك هنا
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=callbacks,
        verbose=1
    )

    # --- 4. FINAL SAVING ---
    model.save('plant_disease_resnet50_final.keras')
    print("✅ Model saved successfully as plant_disease_resnet50_final.keras")

### Evaluate

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


# 1. Evaluate on Test Set
print("\n--- Evaluating on Test Data ---")
results = model.evaluate(test_ds)

# تفكيك النتائج بناءً على ترتيب الـ metrics في الـ compile
test_loss = results[0]
test_acc = results[1]
test_precision = results[2]
test_recall = results[3]

# حساب الـ F1-Score يدوياً
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall + 1e-7)

print(f"✅ Test Accuracy:  {test_acc*100:.2f}%")
print(f"✅ Test Precision: {test_precision*100:.2f}%")
print(f"✅ Test Recall:    {test_recall*100:.2f}%")
print(f"✅ Test F1-Score:  {test_f1*100:.2f}%")
print(f"📉 Test Loss:      {test_loss:.4f}")


# 2. Plotting Training Results (Accuracy, Loss, and Precision/Recall)
plt.figure(figsize=(18, 5))

# Plot Accuracy
plt.subplot(1, 3, 1)
plt.plot(history_fine.history['accuracy'], label='Train Accuracy', color='blue')
plt.plot(history_fine.history['val_accuracy'], label='Val Accuracy', color='orange')
plt.title('Accuracy Evolution')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()

# Plot Loss
plt.subplot(1, 3, 2)
plt.plot(history_fine.history['loss'], label='Train Loss', color='red')
plt.plot(history_fine.history['val_loss'], label='Val Loss', color='darkred')
plt.title('Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Precision & Recall
plt.subplot(1, 3, 3)
plt.plot(history_fine.history['precision'], label='Train Precision', linestyle='--')
plt.plot(history_fine.history['recall'], label='Train Recall', linestyle='--')
plt.title('Precision & Recall Evolution')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()

plt.tight_layout()
plt.show()

# ملخص الموديل النهائي
model.summary()